# Testing the Deck Loading Functionality

Import dependencies

In [1]:
import re
import pandas as pd
from typing import List, Dict, Optional, Any, Set, Callable, Union

from MTG_Engine_v5 import *

### Define Classes

In [8]:
class DeckLoader:
    """Handles the entire process of reading, validating, and loading a deck, including DFCs."""
    def __init__(self, cards_path: str, legalities_path: str):
        self._cards_path = cards_path
        self._legalities_path = legalities_path # Stored for use in load_deck
        self._banned_uuids: Set[str] = set()

    def _clean_name(self, raw: str) -> str:
        # Precompiled patterns for trailing junk
        SETCODE_RE   = re.compile(r'\s*[A-Z]{2,6}[- ]?\d+[a-z]?\s*$', re.ASCII)  # e.g., "KTK-34", "MH1 164", "58p"
        PARENS_RE    = re.compile(r'\s*\([^)]*\)\s*$', re.ASCII)                 # e.g., "(PLST)", "(NEO)", "(6ED)"
        STAR_TAG_RE  = re.compile(r'\s*\*[^*]*\*\s*$', re.ASCII)                 # e.g., "*F*", "*Foil*"
        NUMBER_RE    = re.compile(r'\s*\d+[a-z]?\s*$', re.ASCII)                 # e.g., "15", or "58p" if no set code present

        name = raw.strip()
        # Iteratively strip trailing tokens until nothing more matches
        changed = True
        while changed:
            changed = False
            for pat in (STAR_TAG_RE, SETCODE_RE, NUMBER_RE, PARENS_RE):
                new = pat.sub('', name)
                if new != name:
                    name = new.strip()
                    changed = True
        return name

    def _read_decklist(self, filepath: str) -> Dict[str, int]:
        counts: Dict[str, int] = {}
        commander_name = ""
        first_pass = True # To keep track of first line (commander)

        with open(filepath, 'r', encoding='utf-8') as f:
            for line in f:
                s = line.strip()
                if not s:
                    continue

                # Quantity (handles "4", "4x", "4 x"); default to 1 if missing
                m = re.match(r'^\s*(\d+)\s*x?\s*(.+)$', s, flags=re.IGNORECASE)
                if m:
                    qty, rest = int(m.group(1)), m.group(2)
                else:
                    qty, rest = 1, s

                name = self._clean_name(rest)
                if not name:
                    continue

                # Grab commander name
                if first_pass:
                    commander_name = name
                    first_pass = False

                counts[name] = counts.get(name, 0) + qty

        
        return commander_name, counts

    def _fetch_card_data(self, decklist: Dict[str, int]):
        """Retrieves data for all faces of a specific list of card names."""
        names = decklist.keys()

        chunk_iter_a = pd.read_csv(self._cards_path, chunksize=5000, low_memory=True)
        all_cards_df = pd.concat(
            [chunk[chunk['name'].isin(names)] for chunk in chunk_iter_a]
        )

        # There may be a more efficient way to do this... but the cards db is precleaned except for some DFCs, this grabs only the first 2 occurrences
        all_cards_df = all_cards_df[
            (all_cards_df.groupby("name").cumcount() < 2)
            & ((all_cards_df["otherFaceIds"].notna())
            | (all_cards_df["otherFaceIds"] != ""))
        ]

        # Now duplicate cards that need it
        for name, count in decklist.items():
            if count > 1:
                row = all_cards_df.loc[(all_cards_df["name"] == name) 
                    & ((all_cards_df["otherFaceIds"].isna())
                    | (all_cards_df["otherFaceIds"] == "")) # Ignore DFCs for now (if adding other gamemodes where duplicates are allowed, would need a more sophisticated implementation)
                ].head(1).copy()

                if row.empty: continue # If nothing, continue

                dupes = pd.concat([row] * (count - 1), ignore_index=True) # Create duplicates
                all_cards_df = pd.concat([all_cards_df, dupes], ignore_index=True) # Concat

        # Check if we could find all cards
        for name in names:
            if not name in all_cards_df['name'].values:
                print(f"Error: Couldn't find card {name}")

        return all_cards_df

    def load_deck(self, filepath: str) -> Optional['Deck']:
        print(f"\n--- Loading deck: {filepath} ---")
        commander_name, decklist = self._read_decklist(filepath)

        if commander_name == "":
            return "Error getting commander name."

        # Fetch all required card data first
        all_faces_df = self._fetch_card_data(decklist)

        # Create a Card object for every face, keyed by UUID
        all_faces = [Card(row._asdict()) for row in all_faces_df.itertuples(index=False)]

        # Find commander
        commander = None
        for card in all_faces:
            if card.name == commander_name:
                commander = card
                break

        # Build the final list of cards
        deck = []
        seen = {}

        deck.append(commander) # Put commander first
        all_faces.remove(commander)

        for card in all_faces:
            if not card.is_dfc:
                deck.append(card)
            elif card.name not in seen: # again, this is pretty commander specific, other gamemodes would require more robust system
                deck.append(card)
                seen[card.name] = card
            else:
                seen_card = seen[card.name]
                old_spot = deck.index(seen_card)

                if seen_card.side == 'a':
                    deck[old_spot].otherFace = card
                else:
                    deck[old_spot] = card
                    card.otherFace = seen_card
        
        # Check for missing cards (e.g., typos in decklist)
        if len(deck) != 100:
            print(f"Error: Found {len(deck)} valid cards, but 100 are required.")
            return None

        # --- Validation Logic ---
        commander = deck[0]
        library = deck[1:]

        if not commander.can_be_commander:
            print(f"Validation Error: Card '{commander.name}' is an invalid commander.")
        
        commander_identity = set(commander.colorIdentity)
        seen = [commander]
        for card in library:
            card_identity = set(card.colorIdentity)
            if not card_identity.issubset(commander_identity):
                print(f"Validation Error: Card '{card.name}' is outside the commander's color identity.")
                return None
            
            # Commander cards uniqueness check
            if card in seen and not card.is_basic_land:
                print(f"Validation Error: Card '{card}' is in the deck more than once.")
            else:
                seen.append(card)

        
        print(f"Deck '{filepath}' loaded and validated successfully.")
        return Deck(commander=commander, library=library)

### Try to Load Deck

In [9]:
deckloader = DeckLoader('cards.csv', 'cardLegalities.csv')
deck = deckloader.load_deck('Kinnan_MidPower.txt')


--- Loading deck: Kinnan_MidPower.txt ---
Deck 'Kinnan_MidPower.txt' loaded and validated successfully.
